In [ ]:
from fastlite import database

db = database('scraper.db')

In [ ]:
db.t

chunks, discovered_urls, embeddings, extracts, pages, sites

In [ ]:
db.q(f"select * from sites")

[{'id': 1,
  'root_url': 'https://www.jea.com',
  'selector': '#secondary-content > div > div.cb-content-container.cf',
  'breadcrumb_selector': '#secondary-content > nav',
  'split_function': 'split_md_sections',
  'name': 'JEA'},
 {'id': 2,
  'root_url': 'https://connections',
  'selector': '#post > div.doc-scrollable.editor-content',
  'breadcrumb_selector': 'body > section.page_breadcrumb > div > div > div.col-sm-8.col-md-9 > nav > ol',
  'split_function': 'split_with_tabs',
  'name': 'connections'}]

In [ ]:
db.t.discovered_urls(limit=1)

[{'id': 1,
  'site_id': 1,
  'url': 'https://www.jea.com',
  'discovered_at': '2026-01-24T21:36:45.404252'}]

In [ ]:
def scrape_discovered_pages(db, site_id=None, url_filter=None, delay=0.5):
    """
    Scrape pages from discovered_urls table and store them in pages table.
    
    Args:
        db: Database connection
        site_id: Optional site_id to filter by. If None, scrapes all sites.
        url_filter: Optional callable that takes a URL and returns True if it should be scraped.
        delay: Delay between requests in seconds.
    
    Returns:
        Number of pages scraped.
    """
    import httpx
    import hashlib
    from datetime import datetime
    import time
    
    # Find URLs from discovered_urls that haven't been scraped yet (not in pages table)
    if site_id:
        query = """
            SELECT d.id, d.site_id, d.url 
            FROM discovered_urls d
            LEFT JOIN pages p ON d.url = p.url
            WHERE d.site_id=? AND p.id IS NULL
        """
        params = (site_id,)
    else:
        query = """
            SELECT d.id, d.site_id, d.url 
            FROM discovered_urls d
            LEFT JOIN pages p ON d.url = p.url
            WHERE p.id IS NULL
        """
        params = ()
    
    pages_to_scrape = db.execute(query, params).fetchall()
    
    if url_filter:
        pages_to_scrape = [(pid, sid, url) for pid, sid, url in pages_to_scrape if url_filter(url)]
    
    scraped = 0
    for discovered_id, site_id_val, url in pages_to_scrape:
        try:
            resp = httpx.get(url, timeout=10, follow_redirects=True, verify=False)
            if resp.status_code != 200:
                print(f"✗ {url}: status {resp.status_code}")
                continue
            
            html = resp.text
            content_hash = hashlib.md5(html.encode()).hexdigest()
            now = datetime.utcnow().isoformat()
            
            # Check if we already have this content_hash (duplicate HTML)
            existing_with_hash = list(db.t.pages.rows_where('content_hash=?', [content_hash], limit=1))
            
            if existing_with_hash:
                # Duplicate HTML found - reuse the existing HTML to avoid storing duplicate
                existing_page = existing_with_hash[0]
                existing_html = existing_page['html']
                print(f"⊘ {url} (duplicate HTML, reusing from {existing_page['url']})")
                # Still insert a page record for this URL, but reuse the HTML
                db.t.pages.insert(site_id=site_id_val, url=url, html=existing_html, content_hash=content_hash,
                                 last_scraped=now, last_changed=existing_page.get('last_changed', now))
            else:
                # New unique HTML content
                db.t.pages.insert(site_id=site_id_val, url=url, html=html, content_hash=content_hash,
                                 last_scraped=now, last_changed=now)
                print(f"✓ {url} (scraped)")
                scraped += 1
            
            time.sleep(delay)
        except Exception as e:
            print(f"✗ {url}: {e}")
    
    return scraped

In [ ]:
import httpx
from bs4 import BeautifulSoup
import hashlib
from datetime import datetime

def fetch_page(db, site_id, url):
    """Fetch a single page and store it"""
    site = db.t.sites[site_id]
    if not site: raise ValueError(f"No site with id {site_id}")
    
    resp = httpx.get(url, timeout=10, follow_redirects=True, verify=False)
    html = resp.text
    content_hash = hashlib.md5(html.encode()).hexdigest()
    now = datetime.utcnow().isoformat()
    
    # Check if URL already exists
    existing_by_url = list(db.t.pages.rows_where('url=?', [url], limit=1))
    
    if existing_by_url:
        page = existing_by_url[0]
        # Check if content changed
        if page['content_hash'] != content_hash:
            db.t.pages.update({'id': page['id'], 'html': html, 'content_hash': content_hash,
                              'last_scraped': now, 'last_changed': now})
            print(f"↻ Updated: {url}")
        else:
            db.t.pages.update({'id': page['id'], 'last_scraped': now})
            print(f"  {url} (unchanged)")
        return page['id']
    else:
        # Check if we already have this content_hash (duplicate HTML)
        existing_with_hash = list(db.t.pages.rows_where('content_hash=?', [content_hash], limit=1))
        
        if existing_with_hash:
            # Duplicate HTML found - reuse the existing HTML
            existing_page = existing_with_hash[0]
            existing_html = existing_page['html']
            print(f"⊘ {url} (duplicate HTML, reusing from {existing_page['url']})")
            row = db.t.pages.insert(site_id=site_id, url=url, html=existing_html, content_hash=content_hash,
                                    last_scraped=now, last_changed=existing_page.get('last_changed', now))
        else:
            # New unique HTML content
            row = db.t.pages.insert(site_id=site_id, url=url, html=html, content_hash=content_hash,
                                    last_scraped=now, last_changed=now)
            print(f"✓ New: {url}")
        return row['id']

In [ ]:
scrape_discovered_pages(db, 1)

/tmp/ipykernel_1309/2877083102.py:52: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  now = datetime.utcnow().isoformat()


✓ https://www.jea.com (scraped)


✓ https://www.jea.com/Outage_Center/Outage_FAQ/ (scraped)


✓ https://www.jea.com/Privacy_and_Disclaimers/ (scraped)


✓ https://www.jea.com/About/Procurement/Become_a_Vendor/ (scraped)


...

✓ https://www.jea.com/Engineering_and_Construction/Water_and_Wastewater_Development/Self-Service_Center/Development_Project_Status/ (scraped)


✓ https://www.jea.com/About/Careers/Culture/ (scraped)


✓ https://www.jea.com/About/Procurement/Contractor_Safety/ (scraped)


✓ https://www.jea.com/Engineering_and_Construction/Service_Availability_Form/ (scraped)


10

In [ ]:
# JEA page (site_id=1)
fetch_page(db, 1, 'https://www.jea.com/my_account/rates/')

✓ New: https://www.jea.com/my_account/rates/


/tmp/ipykernel_1309/2159609341.py:14: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  now = datetime.utcnow().isoformat()


11

In [ ]:
# Connections page (site_id=2) 
fetch_page(db, 2, 'https://connections/?docs=residential/start-stop-transfer-traditional-service/transfer-service/transferring-service')

ConnectError: [Errno -2] Name or service not known